In [1]:

import sys
import time
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import time
from pandas import Series, DataFrame

GDriveJSON = 'PythonUpload.json'
GSpreadSheet = 'stock'
stock_code = '3545'
try:
    scope =['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    key = SAC.from_json_keyfile_name(GDriveJSON, scope)
    gc = gspread.authorize(key)
    worksheet = gc.open(GSpreadSheet).worksheet(stock_code)
except Exception as ex:
    print('無法連線Google試算表', ex)
    sys.exit(1)
else:
    data = worksheet.get_all_values()

In [4]:
data = DataFrame(data)
data.columns = data.loc[0]
data = data[1:]

In [5]:
data.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,成交筆數
1,107/05/03,2277731,59023878,26.2,26.4,25.7,25.8,1236
2,107/05/02,1979457,51946823,26.3,26.45,26.1,26.1,1267
3,107/04/30,4512429,117309604,26.5,26.5,25.6,25.95,2506
4,107/04/27,2804362,76649003,27.75,28.25,26.6,27,1362
5,107/04/26,3198776,90291614,28.25,29.2,27.3,27.4,1464


In [26]:
stock_data_df = data.copy()
def replace_year(x):
    if len(x)==9:
        return x.replace(x[0:3],str(int(x[0:3])+1911))
    else:
        return x.replace(x[0:2],str(int(x[0:2])+1911))
stock_data_df['日期'] = stock_data_df['日期'].apply(replace_year)

In [27]:
stock_data_df.index=stock_data_df['日期']
stock_data_df.drop(['日期','成交金額','成交筆數'],axis=1,inplace=True)
stock_data_df.columns=['volume','open','high','low','close']

In [28]:
stock_data_df['ret'] = 0
stock_data_df.loc[stock_data_df['close'] > stock_data_df['open'],'ret'] = 1

In [29]:
stock_data_df

,volume,open,high,low,close,ret
日期,,,,,,
2018/05/03,2277731,26.2,26.4,25.7,25.8,0
2018/05/02,1979457,26.3,26.45,26.1,26.1,0
2018/04/30,4512429,26.5,26.5,25.6,25.95,0
2018/04/27,2804362,27.75,28.25,26.6,27,0
2018/04/26,3198776,28.25,29.2,27.3,27.4,0
2018/04/25,2568804,27.1,28.15,27,28.05,1
2018/04/24,2464453,28.65,28.65,27.25,27.55,0
2018/04/23,2529243,28.3,29.3,28.3,28.65,1
2018/04/20,4936871,28.6,29.5,28.35,28.55,0
